In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import gc

tf.__version__

'1.14.0'

In [2]:
% ls

'355842076730650 ID data : 1'
 checkpoint
 DTC_analysis.ipynb
 DTC_Error_Predictive_Model.ipynb
'DTC Predictive Model v2.ipynb'
 events.out.tfevents.1562224465.parth-VivoBook-ASUSLaptop-X580GD
 tf_logs/
 tf_logs2/


In [3]:
%load_ext tensorboard
%tensorboard --logdir tf_logs2/

Reusing TensorBoard on port 6007 (pid 21058), started 12:28:56 ago. (Use '!kill 21058' to kill it.)

In [4]:
base_dir = '/home/parth/Machine Learning/Datasets/North Corp/DTC'
filePcode = "Top_LMD_DTC_SPN.csv"


file_list = os.listdir(base_dir)
file_list_corrected = []
for file_ in file_list:
    if file_.endswith(".csv") and file_ != filePcode:
        file_list_corrected.append(file_)
file_list = file_list_corrected

In [1]:
index = 0
filename = os.path.join(base_dir , file_list[index])
data = pd.read_csv(filename)
pcode_data = pd.read_csv(os.path.join(base_dir , filePcode))

NameError: name 'os' is not defined

In [6]:
ids = data.deviceID.unique()
device_data = []
for id_ in ids:
    device_data.append(data[data.deviceID == id_].copy())


In [7]:
for i , device_data_ in enumerate(device_data):
    device_data_ = device_data_.reset_index()
    device_data_["spn_fmi"] = None
    for index , (a, b) in enumerate(zip(device_data_["spn"] , device_data_["fmi"])):
        device_data_.at[ index , "spn_fmi"] = (a,b)
    device_data[i] = device_data_


In [8]:
pcode_data["spn_fmi"] = None
for index , (a, b) in enumerate(zip(pcode_data["SPN"] , pcode_data["FMI"])):
    pcode_data.at[ index , "spn_fmi"] = (a,b) 

code2pcode = {}
for code , pcode in  zip(pcode_data["spn_fmi"] , pcode_data["Error codes "]):
    code2pcode[code] = pcode

In [9]:
for i in range(len(device_data)):
    device_data[i]["Pcode"] = None
    for index , spn_fmi_device in enumerate(device_data[i]["spn_fmi"] ):
        if spn_fmi_device in code2pcode.keys():
            device_data[i].at[index , "Pcode"] = code2pcode[spn_fmi_device]

In [10]:
for i in range(len(device_data)):
    device_data[i].sort_values("utc" , inplace=True , ascending=True)

In [11]:
vocab = set()
for i in range(len(device_data)):
    vocab.update(device_data[i]["spn_fmi"].unique())
vocab.update((-1 , -1))

In [12]:
code2id = {}
pcode2class = {}
severe_codes = 0
code2id[(-1 , -1)] = 0
for i , voc in enumerate(vocab):
    code2id[voc] = i + 1
    if voc in code2pcode.keys() :
        pcode = code2pcode[voc]
        pcode2class[pcode] = severe_codes + 1
        severe_codes = severe_codes + 1
id2code = {b:a for a,b in code2id.items()}


In [13]:
code2class = {}
for i , code in enumerate(code2id.keys()):
    code2class[code] = 0#max(pcode2class.values()) + i + 1
    if code in code2pcode:
        code2class[code] = pcode2class[code2pcode[code]]


In [14]:
device_dataMLX = []
device_dataMLy = []
for device in device_data:
    utcs = device.utc.unique()
    temp_deviceData = []
    temp_temp_targets = []
    first = True
    for utc in utcs:
        temp_targets = [0] * (severe_codes)
        temp_list = []
        codes = device[device.utc == utc]["spn_fmi"]
        for code in codes:
            id_ = code2id[code]
            class_ = code2class[code]
            if class_ != 0:
                temp_targets[class_ - 1] = temp_targets[class_ - 1] + 1
            temp_list.append(id_)
        if not first:
            temp_temp_targets.append(temp_targets)
        first = False
        temp_list = np.array(temp_list)
        temp_deviceData.append(temp_list)
    temp_temp_targets.append([0]*severe_codes)
    device_dataMLy.append(temp_temp_targets)
    device_dataMLX.append(temp_deviceData)
device_dataMLy = np.array(device_dataMLy)

In [15]:
X = []
for device in device_dataMLX:
    X_ = np.zeros(shape=(len(device)  , len(vocab)))
    for index , dev in enumerate(device):
        X_[index , dev] = X_[index , dev] + 1
    X.append(X_)

In [16]:
def makeSequences(seq_length):
    seqsX = []
    seqsy = []
    n_seq = 0
    for x in X:
        n_seq = n_seq + (len(x) // seq_length)
    for i , (devX , devY) in enumerate(zip(X , device_dataMLy)):
        for step in range( len(devX) // seq_length ):
            start = step * seq_length
            end = start + seq_length
            seqX , seqy = (devX[start:end] , devY[start:end])
            seqsX.append(seqX)
            seqsy.append(seqy)
    return ( np.array(seqsX) , np.array(seqsy) ) , n_seq

In [17]:
seqs , n_seqs = makeSequences(10)

In [18]:
def makeBatches(seqs , n_seqs , batch_size , random_state = None):
    if random_state is not None:
        np.random.seed(random_state)
    perms = np.random.permutation(n_seqs)
    n_batches = n_seqs // batch_size
    batchesX = np.array_split(seqs[0][perms] , n_batches)
    batchesy = np.array_split(seqs[1][perms] , n_batches)
    return batchesX , batchesy

In [19]:
Xval , yval = None , None

In [20]:
n_neurons = 300
n_layers = 1
input_keep_prob = 1
vocab_size = len(vocab)
compressed_size = 200
print(vocab_size)

2465


In [21]:
batches = makeBatches(seqs , n_seqs , 10)

In [22]:
batches[0]

[array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 1., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],


In [28]:
tf.reset_default_graph()
he_init = tf.variance_scaling_initializer()
with tf.name_scope("Inputs"):
    # Targets - either critical or No case
    targets = tf.placeholder(tf.int32 , shape=[None , None , severe_codes ] , name="targets")
    inputs = tf.placeholder(tf.float32 , shape=[None , None , vocab_size] , name="inputs")
    emphasis = tf.placeholder(tf.float32 , shape=() , name="Emphasis")

with tf.name_scope("Compressed"):
    compressed_inputs = tf.layers.dense(inputs , compressed_size , activation=tf.nn.elu , kernel_initializer=he_init)

with tf.name_scope("RNN"):
    basic_cell = [tf.contrib.rnn.GRUCell(num_units = n_neurons)] * n_layers
    droupout_cell = [tf.contrib.rnn.DropoutWrapper(basic_cell_ , input_keep_prob=input_keep_prob) for basic_cell_ in basic_cell]
    stacked_cells = tf.contrib.rnn.MultiRNNCell(droupout_cell)
    outputs , final_state = tf.nn.dynamic_rnn(stacked_cells , compressed_inputs , dtype=tf.float32)
with tf.name_scope("Hidden_Layers"):
    hidden_layers = tf.layers.dense(outputs , units=50 , activation=tf.nn.elu , kernel_initializer=he_init)
    logits = tf.layers.dense(hidden_layers , severe_codes , activation=None , kernel_initializer=he_init)
with tf.name_scope("Loss"):
    non_zero_mask = tf.logical_not(tf.equal(targets , 0))
    loss = tf.nn.softmax_cross_entropy_with_logits(
        labels = targets,
        logits = logits
    )  
    #mask =  emphasis  * tf.cast(tf.logical_not(tf.equal(target_errors_placeholder  , 0)) , tf.float32)# + tf.ones_like(loss) )
    #loss = loss * mask
    loss = tf.reduce_mean(loss)
with tf.name_scope("Evaluation"):
    predictions = tf.cast( logits > 0, tf.int32)
    accuracy = tf.reduce_mean(tf.cast( tf.equal( tf.boolean_mask(predictions , non_zero_mask) , tf.boolean_mask(targets , non_zero_mask)) , tf.float32))
with tf.name_scope("Training"):
    optimizer = tf.train.AdamOptimizer()
    gradients = optimizer.compute_gradients(loss , var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
    #gradients = [ (tf.clip_by_value(var[0] , clip_value_max=1.5 , clip_value_min=-1.5)  , var[1] ) for var in gradients]
    training_op = optimizer.apply_gradients(gradients)
    init = tf.global_variables_initializer()

with tf.name_scope("Logs"):
    file_writer = tf.summary.FileWriter("tf_logs2/")
    acc_sum = tf.summary.scalar("Accuracy_Summary" , accuracy)
    loss_sum = tf.summary.scalar("Loss_Summary" , loss)
    saver = tf.train.Saver()
#    return graph

In [22]:
n_epoch = 200
batch_size = 10
n_batches = len(code2class)
evaluation_gap = 1
maxcheckswithoutprogress = 2000
savedir = "."

max_batch_size = 20

In [23]:
def get_model_params(sess):
    return { gvar.op.name : sess.run(gvar) for gvar in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)}

def restore_model_params(sess , params):
    gvar_names = params.keys()
    assign_ops = {gvar : tf.get_default_graph().get_operation_by_name(gvar + "/Assign") for gvar in gvar_names}
    init_values = {gvar : assign_ops[gvar].inputs[1] for gvar in gvar_names}
    for gvar in gvar_names:
        sess.run(assign_ops[gvar] , feed_dict={init_values[gvar] : params[gvar]})

In [24]:
from IPython import display

In [25]:

with tf.Session() as sess:
    file_writer.add_graph(tf.get_default_graph())

In [29]:

with tf.Session() as sess:
    file_writer.add_graph(tf.get_default_graph())
    accuracy_val = 0
    try:
        restore_model_params(sess , best_params)
    except NameError:
        bbest_params = None
        sess.run(init)
    global_step = 0
    feed_dict_val = None
    if Xval is not None and yval is not None :
        feed_dict_val = {inputs : Xval , target_reviews : yval , input_seq_len : seq_val}
    best_yet = 0
    checkswithoutprogress = 0
    for epoch in range(n_epoch):
        step = 0
        batchesX , batchesy = makeBatches(seqs , n_seqs , batch_size=batch_size)
        for Xbatch , ybatch in zip(batchesX , batchesy):
            print("Epoch : {} Step : {}".format(epoch , step + 1))
            step = step + 1
            global_step = global_step + 1
            feed_dict = {
                inputs : Xbatch ,
                targets : ybatch ,
                #emphasis : 10
                        }
            _ , loss_summary = sess.run([training_op , loss_sum] , feed_dict = feed_dict)
            file_writer.add_summary( loss_summary , global_step)
            if global_step % evaluation_gap == 0 :
                if feed_dict_val is None :
                    feed_dict_val = feed_dict
                accuracy_val  , _ = sess.run([accuracy , acc_sum] , feed_dict=feed_dict_val)
                file_writer.add_summary( _ , global_step)
                display.clear_output(wait = True)
                print("Accuracy : {}".format(accuracy_val))
                if accuracy_val > best_yet:
                    best_yet = accuracy_val
                    best_params = get_model_params(sess)
                    checkswithoutprogress = 0
                else :
                    checkswithoutprogress = checkswithoutprogress + 1
            if checkswithoutprogress > maxcheckswithoutprogress :
                print("Epoch {} Accuracy {} [Incomplete epoch , No progress for {} steps]".format(epoch + 1 , accuracy_val , checkswithoutprogress))
                break
        else :
            print("Epoch {} Accuracy {} ".format(epoch + 1 , accuracy_val))
            continue
        break
    restore_model_params(sess , best_params)
    saver.save(sess , savedir)

Accuracy : 0.0
Epoch 72 Accuracy 0.0 [Incomplete epoch , No progress for 2001 steps]
